### アイデア
次の二乗誤差を予測することである種の分布を得ようという試み

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime

### データの読み込み

In [ ]:
def load_data():
    """
    return X_train, X_test, y_train, y_test
    """
    print("loading the data...")
    DATA_DIR="./data/"
    X_train = np.load(os.path.join(DATA_DIR, "X_train.npy"))
    print("X train",X_train.shape)
    X_test = np.load(os.path.join(DATA_DIR, "X_test.npy"))
    print("X test",X_test.shape)
    y_train = np.load(os.path.join(DATA_DIR, "y_train.npy"))
    print("y train",y_train.shape)
    y_test = np.load(os.path.join(DATA_DIR, "y_test.npy"))
    print("y test",y_test.shape)
    
    # shapeをglobal変数に
    global NUM_timesteps, NUM_input_dim, NUM_output_dim
    _, NUM_timesteps, NUM_input_dim = X_train.shape
    _, NUM_output_dim = y_train.shape
    
    
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = load_data()

### モデルの作成
まずは平均値を学習するモデルを作成

In [ ]:
from keras.layers import Dense, RepeatVector, Input, Lambda
from keras.layers import GRU
# from keras.layers import CuDNNGRU as GRU #GPU用
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.optimizers import adam
from keras import backend as K
from keras.models import Model

In [ ]:
def ret_model(est="mu"):
    """
    GRU
    FC
    FC
    mu
    """
    
    
    # hyper parameter
    LATENT = 20
    FC=10
    
    # ネットワークの定義
    inputs = Input(shape=(NUM_timesteps, NUM_input_dim))
    # (, NUM_timesteps, NUM_input_dim)
    gru=GRU(LATENT)(inputs)
    # (, LATENT)
    fc=Dense(FC,activation="relu")(gru)
    # (, FC)
    if est is not "mu":
        output=Dense(NUM_output_dim,activation = "sigmoid")(fc)
    else:
        output=Dense(NUM_output_dim)(fc)
    
    model = Model(inputs,output)
    
    model.summary()
    
    return model       

In [ ]:
mu_predictor = ret_model()
mu_predictor.compile(optimizer=adam(lr=0.001),loss="mean_squared_error")

### 訓練

In [ ]:
logdir="./"+str(datetime.now().strftime('%s'))+"mu/"
mu_predictor.fit(X_train, y_train,
         epochs=20,
         batch_size=128,
         shuffle=True,
         validation_split=0.1,
         callbacks=[TensorBoard(log_dir=logdir), 
                   EarlyStopping(patience=2),
                   ModelCheckpoint(filepath = logdir+'model_epoch.{epoch:02d}-los{val_loss:.4f}.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='auto')])

### 誤差を取る

In [ ]:
temp=mu_predictor.predict(X_train)
err_var=np.square(temp-y_train)
err_var.shape

In [ ]:
plt.plot(err_var[:192])

### この系列を学習させる
学習データから二乗誤差の遷移を学習する

In [ ]:
var_predictor=ret_model(est="var")
var_predictor.compile(optimizer=adam(lr=0.001),loss="mean_squared_error")

In [ ]:
logdir="./"+str(datetime.now().strftime('%s'))+"var/"
var_predictor.fit(X_train, err_var,
         epochs=20,
         batch_size=128,
         shuffle=True,
         validation_split=0.1,
         callbacks=[TensorBoard(log_dir=logdir), 
                   EarlyStopping(patience=2),
                   ModelCheckpoint(filepath = logdir+'model_epoch.{epoch:02d}-los{val_loss:.4f}.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='auto')])

### 結果の可視化

In [ ]:
mu = mu_predictor.predict(X_test)
sigma = np.abs(var_predictor.predict(X_test)) #ここではまだσ^2
sigma = np.sqrt(sigma)

In [ ]:
upper = mu+sigma
lower = mu-sigma
upper.shape, lower.shape

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(mu[:192])
#plt.plot(upper[:192,0])
plt.fill_between(range(192),upper[:192,0],lower[:192,0],color="green",alpha=0.2)
plt.plot(y_test[:192])

### 訓練済みモデルを使った一連の推論

In [ ]:
from keras.models import load_model

mu_path="./1539513731mu/model_epoch.08-los0.0025.h5"
var_path="./1539514907var/model_epoch.02-los0.0000.h5"


In [ ]:
mu_predictor = load_model(mu_path)
var_predictor = load_model(var_path)

In [ ]:
mu = mu_predictor.predict(X_test)
sigma = np.abs(var_predictor.predict(X_test)) #ここではまだσ^2
sigma = np.sqrt(sigma)

upper = mu+2*sigma
lower = mu-2*sigma
upper.shape, lower.shape

In [ ]:
def show_interval(start,width=192):
    plt.figure(figsize=(8,6))
    plt.plot(mu[start:start+width], color="g", alpha=0.3)
    #plt.plot(upper[:192,0])
    plt.fill_between(range(width),upper[start:start+width,0],lower[start:start+width,0],color="green",alpha=0.2)
    plt.plot(y_test[start:start+width])
    plt.show()
def show_sigma(start,width=192):
    plt.figure(figsize=(8,6))
    plt.plot(sigma[start:start+width],color="g")
#     plt.plot(0.06*y_test[start:start+width]+0.06)
    plt.show()

In [ ]:
for i in range(0,len(y_test),192):
    show_interval(i)

In [ ]:
show_sigma(0,width=96*2)